We quickly tested Tensorflow sample MNIST GDO / SOFTMAX Classifier

In [4]:
import pandas as pd
import input_data
import numpy as np

In [19]:
training_target = np.load('data/target.pkl')
training_data = np.load('data/training.pkl')

In [21]:
training_data.shape

(14097, 39)

In [20]:
training_target.shape

(14097,)

In [43]:
def dense_to_one_hot(labels_dense, num_classes=2):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

class DataSet(object):
  def __init__(self, samples, labels, fake_data=False):
    if fake_data:
      self._num_examples = 10000
    else:
      assert samples.shape[0] == labels.shape[0], (
          "samples.shape: %s labels.shape: %s" % (samples.shape,
                                                 labels.shape))
      self._num_examples = samples.shape[0]
      # Convert shape from [num examples, rows, columns, depth]
      # to [num examples, rows*columns] (assuming depth == 1)
      '''
      assert samples.shape[3] == 1
      samples = samples.reshape(samples.shape[0],
                              samples.shape[1] * samples.shape[2])
      # Convert from [0, 255] -> [0.0, 1.0].
      samples = images.astype(numpy.float32)
      samples = numpy.multiply(samples, 1.0 / 255.0)
      '''

    self._samples = samples
    self._labels = dense_to_one_hot(labels)
    self._epochs_completed = 0
    self._index_in_epoch = 0
  @property
  def samples(self):
    return self._samples
  @property
  def labels(self):
    return self._labels
  @property
  def num_examples(self):
    return self._num_examples
  @property
  def epochs_completed(self):
    return self._epochs_completed
  def next_batch(self, batch_size, fake_data=False):
    """Return the next `batch_size` examples from this data set."""
    if fake_data:
      fake_image = [1.0 for _ in xrange(self._samples.shape[1])]
      fake_label = 0
      return [fake_image for _ in xrange(batch_size)], [
          fake_label for _ in xrange(batch_size)]
    start = self._index_in_epoch
    self._index_in_epoch += batch_size
    if self._index_in_epoch > self._num_examples:
      # Finished epoch
      self._epochs_completed += 1
      # Shuffle the data
      perm = np.arange(self._num_examples)
      np.random.shuffle(perm)
      self._samples = self._samples[perm]
      self._labels = self._labels[perm]
      # Start next epoch
      start = 0
      self._index_in_epoch = batch_size
      assert batch_size <= self._num_examples
    end = self._index_in_epoch
    return self._samples[start:end], self._labels[start:end]


In [49]:
import tensorflow as tf

In [50]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 1000
display_step = 1

In [51]:
# tf Graph Input
x = tf.placeholder("float", [None, X_training.shape[1]]) # mnist data image of shape 28*28=784
y = tf.placeholder("float", [None, 2]) # 0-9 digits recognition => 10 classes

In [52]:
# Create model

# Set model weights
W = tf.Variable(tf.zeros([X_training.shape[1], 2]))
b = tf.Variable(tf.zeros([2]))

In [53]:
# Construct model
activation = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

In [54]:
# Minimize error using cross entropy
# Cross entropy
cost = -tf.reduce_sum(y*tf.log(activation)) 
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost) 

In [55]:
# Initializing the variables
init = tf.initialize_all_variables()

In [56]:

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = train.next_batch(batch_size)
            # Fit training using batch data
            sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys})
            # Compute average loss
            avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print( "Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(activation, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: test.samples, y: test.labels}))

Epoch: 0001 cost= nan
Epoch: 0002 cost= nan
Epoch: 0003 cost= nan
Epoch: 0004 cost= nan
Epoch: 0005 cost= nan
Epoch: 0006 cost= nan
Epoch: 0007 cost= nan
Epoch: 0008 cost= nan
Epoch: 0009 cost= nan
Epoch: 0010 cost= nan
Epoch: 0011 cost= nan
Epoch: 0012 cost= nan
Epoch: 0013 cost= nan
Epoch: 0014 cost= nan
Epoch: 0015 cost= nan
Epoch: 0016 cost= nan
Epoch: 0017 cost= nan
Epoch: 0018 cost= nan
Epoch: 0019 cost= nan
Epoch: 0020 cost= nan
Epoch: 0021 cost= nan
Epoch: 0022 cost= nan
Epoch: 0023 cost= nan
Epoch: 0024 cost= nan
Epoch: 0025 cost= nan
Optimization Finished!
Accuracy: 0.599079
